# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
# import csv and load into a DataFrame

city_weather_path = "../WeatherPy/output_data/city_weather_clean.csv"

city_weather = pd.read_csv(city_weather_path)

city_weather.head()



,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,hermanus,-34.42,19.23,42.008,93.0,69.0,5.45828,ZA,08-18-2020
1,port alfred,-33.59,26.89,54.914,67.0,100.0,30.33372,ZA,08-18-2020
2,takab,36.40,47.11,60.674,63.0,0.0,5.39117,IR,08-18-2020
3,griffith,41.53,-87.42,71.996,43.0,1.0,11.40870,US,08-18-2020
4,alice springs,-23.70,133.88,60.800,25.0,0.0,9.17170,AU,08-18-2020


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
gmaps.configure(api_key = g_key)

In [5]:
# create a tuple of the lat,longs
city_weather['Location'] = city_weather[['Lat','Lng']].apply(tuple, axis = 1)

In [11]:
# set variables 

locations = city_weather['Location']
humidity = city_weather['Humidity']

In [13]:
# create heat map using humidity as the variable
# use max_intensity = city_weather['Humidity'].max() to use the highest humidity value in dataframe for max intensity

fig = gmaps.figure(center = (48.0,0.0), zoom_level = 1.5)

heat_layer = gmaps.heatmap_layer(
                locations,
                weights = humidity,
                max_intensity = city_weather['Humidity'].max(),
                dissipating = False,
                point_radius = 1.75,
                opacity = 1,
                gradient = None)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [14]:
# find cities that have temps between 70 and 80 degrees, winds less than 10 mph, and 0 cloudiness

ideal_vacation_cities = city_weather[(city_weather['Max Temp'] < 80) & (city_weather['Max Temp'] > 70) & (city_weather['Wind Speed'] < 10) & (city_weather['Cloudiness'] == 0)]

In [15]:
ideal_vacation_cities

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Location
13,caravelas,-17.71,-39.25,73.058,84.0,0.0,8.32164,BR,08-18-2020,"(-17.71, -39.25)"
15,sinnamary,5.38,-52.95,78.746,85.0,0.0,8.09794,GF,08-18-2020,"(5.38, -52.95)"
207,esmeraldas,-19.76,-44.31,73.400,60.0,0.0,8.05320,BR,08-19-2020,"(-19.76, -44.31)"
248,marrakesh,31.63,-8.01,78.800,54.0,0.0,6.93470,MA,08-18-2020,"(31.63, -8.01)"
367,vrangel,42.73,133.08,74.894,83.0,0.0,5.88331,RU,08-19-2020,"(42.73, 133.08)"
371,zaozerne,45.16,33.28,70.754,67.0,0.0,4.38452,UA,08-19-2020,"(45.16, 33.28)"
408,suez,29.97,32.53,73.166,80.0,0.0,7.53869,EG,08-19-2020,"(29.97, 32.53)"
421,daru,-9.08,143.21,78.368,64.0,0.0,9.82043,PG,08-19-2020,"(-9.08, 143.21)"
457,buritizeiro,-17.35,-44.96,74.120,48.0,0.0,6.71100,BR,08-19-2020,"(-17.35, -44.96)"
483,gat,31.61,34.76,77.000,92.0,0.0,4.00423,IL,08-19-2020,"(31.61, 34.76)"


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [16]:
# make a copy (was getting an error) and add column with header 'Hote Name'
hotel_df = ideal_vacation_cities.copy()

hotel_df['Hotel Name'] = ''

In [17]:
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Location,Hotel Name
13,caravelas,-17.71,-39.25,73.058,84.0,0.0,8.32164,BR,08-18-2020,"(-17.71, -39.25)",
15,sinnamary,5.38,-52.95,78.746,85.0,0.0,8.09794,GF,08-18-2020,"(5.38, -52.95)",
207,esmeraldas,-19.76,-44.31,73.400,60.0,0.0,8.05320,BR,08-19-2020,"(-19.76, -44.31)",
248,marrakesh,31.63,-8.01,78.800,54.0,0.0,6.93470,MA,08-18-2020,"(31.63, -8.01)",
367,vrangel,42.73,133.08,74.894,83.0,0.0,5.88331,RU,08-19-2020,"(42.73, 133.08)",
371,zaozerne,45.16,33.28,70.754,67.0,0.0,4.38452,UA,08-19-2020,"(45.16, 33.28)",
408,suez,29.97,32.53,73.166,80.0,0.0,7.53869,EG,08-19-2020,"(29.97, 32.53)",
421,daru,-9.08,143.21,78.368,64.0,0.0,9.82043,PG,08-19-2020,"(-9.08, 143.21)",
457,buritizeiro,-17.35,-44.96,74.120,48.0,0.0,6.71100,BR,08-19-2020,"(-17.35, -44.96)",
483,gat,31.61,34.76,77.000,92.0,0.0,4.00423,IL,08-19-2020,"(31.61, 34.76)",


In [18]:
# set parameters for google nearby search

params = {
    'radius' : 5000,
    'type' : 'lodging',
    'key' : g_key
}

In [19]:
# loop through locations to pull near by hotels and save in 'Hotel Name'

for index, row in hotel_df.iterrows():
    lat = row['Lat']
    long = row['Lng']
    
    params['location'] = f"{lat},{long}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    hotel_request = requests.get(base_url, params = params).json()
    
    try:
        hotel_df.loc[index, 'Hotel Name'] = hotel_request['results'][0]['name']
        
    except (KeyError, IndexError):
        print("Hotel not found. Skipping......")

In [23]:
# save to csv
hotel_df.to_csv("output_data/hotels_in_ideal_cities.csv", index = False)

hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Location,Hotel Name
13,caravelas,-17.71,-39.25,73.058,84.0,0.0,8.32164,BR,08-18-2020,"(-17.71, -39.25)",Pousada dos Navegantes
15,sinnamary,5.38,-52.95,78.746,85.0,0.0,8.09794,GF,08-18-2020,"(5.38, -52.95)",ALETHEIA Appartement Sinnamary
207,esmeraldas,-19.76,-44.31,73.400,60.0,0.0,8.05320,BR,08-19-2020,"(-19.76, -44.31)",Hotel Fazenda Lagoa Azul
248,marrakesh,31.63,-8.01,78.800,54.0,0.0,6.93470,MA,08-18-2020,"(31.63, -8.01)",Les Jardins de La Koutoubia
367,vrangel,42.73,133.08,74.894,83.0,0.0,5.88331,RU,08-19-2020,"(42.73, 133.08)","Отель ""БАЛИ"""
371,zaozerne,45.16,33.28,70.754,67.0,0.0,4.38452,UA,08-19-2020,"(45.16, 33.28)","Molodozhnyy Lager' ""Pioner"""
408,suez,29.97,32.53,73.166,80.0,0.0,7.53869,EG,08-19-2020,"(29.97, 32.53)",Green House
421,daru,-9.08,143.21,78.368,64.0,0.0,9.82043,PG,08-19-2020,"(-9.08, 143.21)",New century hotel
457,buritizeiro,-17.35,-44.96,74.120,48.0,0.0,6.71100,BR,08-19-2020,"(-17.35, -44.96)",Santos Palace Hotel
483,gat,31.61,34.76,77.000,92.0,0.0,4.00423,IL,08-19-2020,"(31.61, 34.76)",רגע בנחת


In [21]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [22]:
# Add marker layer ontop of heat map

marker_layer = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig = gmaps.figure()
fig.add_layer(marker_layer)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))